In [37]:
import os
import glob
import numpy as np
import pandas as pd


In [2]:
# Upload zeek_logs.zip (or zeek_logs (1).zip) and unpack into ./zeek_logs

LOG_DIR = "zeek_logs"

# Try Colab upload first
zip_name = None
try:
    from google.colab import files
    print("📁 Please upload your zeek_logs.zip file...")
    uploaded = files.upload()
    # Take the first uploaded file (e.g., 'zeek_logs.zip' or 'zeek_logs (1).zip')
    zip_name = list(uploaded.keys())[0]
    print("✅ Uploaded:", zip_name)
except Exception as e:
    print("Not in Colab or upload not available. Make sure the ZIP is in this folder.")
    # Fallback default name
    zip_name = "zeek_logs.zip"

# Unzip into ./zeek_logs
print("\n📦 Unzipping logs...")
os.system(f"rm -rf {LOG_DIR}")
os.system(f'unzip -o "{zip_name}" -d {LOG_DIR} > /dev/null')

print("\n📂 Top-level LOG_DIR contents:")
top_contents = glob.glob(os.path.join(LOG_DIR, "*"))
for c in top_contents:
    print("  -", c)

# If zeek_logs/ has exactly one subdirectory (like zeek_logs/zeek_logs), dive into it
subdirs = [d for d in top_contents if os.path.isdir(d)]
if len(subdirs) == 1:
    LOG_DIR = subdirs[0]
    print("\n🔧 Adjusted LOG_DIR to inner folder:", LOG_DIR)
else:
    print("\nLOG_DIR remains:", LOG_DIR)

print("\n📂 Final LOG_DIR contents:")
for c in glob.glob(os.path.join(LOG_DIR, "*")):
    print("  -", c)


📁 Please upload your zeek_logs.zip file...


Saving zeek_logs.zip to zeek_logs.zip
✅ Uploaded: zeek_logs.zip

📦 Unzipping logs...

📂 Top-level LOG_DIR contents:
  - zeek_logs/zeek_logs

🔧 Adjusted LOG_DIR to inner folder: zeek_logs/zeek_logs

📂 Final LOG_DIR contents:
  - zeek_logs/zeek_logs/analyzer.log
  - zeek_logs/zeek_logs/ocsp.log
  - zeek_logs/zeek_logs/packet_filter.log
  - zeek_logs/zeek_logs/stats.log
  - zeek_logs/zeek_logs/files.log
  - zeek_logs/zeek_logs/kerberos.log
  - zeek_logs/zeek_logs/notice.log
  - zeek_logs/zeek_logs/sip.log
  - zeek_logs/zeek_logs/conn.log
  - zeek_logs/zeek_logs/dce_rpc.log
  - zeek_logs/zeek_logs/ntlm.log
  - zeek_logs/zeek_logs/dhcp.log
  - zeek_logs/zeek_logs/dns.log
  - zeek_logs/zeek_logs/x509.log
  - zeek_logs/zeek_logs/ssl.log
  - zeek_logs/zeek_logs/rdp.log
  - zeek_logs/zeek_logs/weird.log
  - zeek_logs/zeek_logs/ntp.log
  - zeek_logs/zeek_logs/known_services.log
  - zeek_logs/zeek_logs/loaded_scripts.log
  - zeek_logs/zeek_logs/known_certs.log
  - zeek_logs/zeek_logs/capture_loss

In [3]:
def load_zeek_log(path: str) -> pd.DataFrame:
    """Load a Zeek tab-separated log (no headers, # comment header lines)."""
    try:
        df = pd.read_csv(path, sep="\t", comment="#", header=None, low_memory=False)
        df["source_log"] = os.path.basename(path)
        return df
    except Exception as e:
        print(f"⚠️ Failed to load {path}: {e}")
        return pd.DataFrame()

def load_many(pattern: str) -> pd.DataFrame:
    """Load all logs matching a pattern under LOG_DIR, concat into one DataFrame."""
    files = glob.glob(os.path.join(LOG_DIR, pattern))
    print(f"\nPattern '{pattern}' → {len(files)} file(s):", [os.path.basename(f) for f in files])
    dfs = []
    for f in files:
        df = load_zeek_log(f)
        if not df.empty:
            dfs.append(df)
    if not dfs:
        return pd.DataFrame()
    return pd.concat(dfs, ignore_index=True)

# Adjust ssl pattern to "tls*.log" if your logs are named tls.log instead of ssl.log
conn_df = load_many("conn*.log")
ssl_df  = load_many("ssl*.log")   # change to "tls*.log" if needed
x509_df = load_many("x509*.log")

print("\nRows loaded:")
print("  conn_df:", len(conn_df))
print("  ssl_df :", len(ssl_df))
print("  x509_df:", len(x509_df))

conn_df.head()



Pattern 'conn*.log' → 1 file(s): ['conn.log']

Pattern 'ssl*.log' → 1 file(s): ['ssl.log']

Pattern 'x509*.log' → 1 file(s): ['x509.log']

Rows loaded:
  conn_df: 1492155
  ssl_df : 419903
  x509_df: 1278


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,source_log
0,1.518697e+09,CnIDHq42unWALrSlp2,175.195.219.31,54576,172.31.69.30,3389,tcp,-,1.607211,746,...,T,0,DdaR,3,866,3,450,-,6,conn.log
1,1.518697e+09,CfDSEFzzaZlJPcG55,201.92.155.17,44655,172.31.68.21,81,tcp,-,-,-,...,T,0,S,1,44,0,0,-,6,conn.log
2,1.518697e+09,CmdO0d4pOoxS3JGAHh,175.195.219.31,55135,172.31.66.122,3389,tcp,-,2.328895,1085,...,T,0,DdaR,5,1285,5,1762,-,6,conn.log
3,1.518697e+09,C11P8c2rLsp4ZfuEd,172.31.66.122,49688,205.186.147.63,443,tcp,-,0.011620,0,...,F,0,Sr,1,52,1,40,-,6,conn.log
4,1.518697e+09,CA9dvAq5ATKNC5wNl,172.31.66.122,49688,205.186.147.63,443,tcp,-,0.011671,0,...,F,0,Sr,1,52,1,40,-,6,conn.log


In [4]:
# --- Fix conn_df column names (based on Zeek conn.log schema + our source_log) ---

conn_cols = [
    "ts",              # 1518697398.261561
    "uid",             # CnIDHq42unWALrSlp2
    "id.orig_h",       # 175.195.219.31
    "id.orig_p",       # 54576
    "id.resp_h",       # 172.31.69.30
    "id.resp_p",       # 3389
    "proto",           # tcp
    "service",         # -
    "duration",        # 1.607211
    "orig_bytes",      # 746
    "resp_bytes",      # 330
    "conn_state",      # OTH
    "local_orig",      # F
    "local_resp",      # T
    "missed_bytes",    # 0
    "history",         # DdaR
    "orig_pkts",       # 3
    "orig_ip_bytes",   # 866
    "resp_pkts",       # 3.1
    "resp_ip_bytes",   # 450
    "tunnel_parents",  # -.1
    "source_log"       # added by loader
]

print("conn_df cols BEFORE:", conn_df.columns.tolist())
if len(conn_df.columns) == len(conn_cols):
    conn_df.columns = conn_cols
else:
    print("⚠️ conn_df column count mismatch; got", len(conn_df.columns), "expected", len(conn_cols))

print("conn_df cols AFTER:", conn_df.columns.tolist())
display(conn_df.head())

# --- Fix ssl_df column names (based on Zeek ssl.log schema + our source_log) ---

ssl_cols = [
    "ts",                      # 1518697405.587349
    "uid",                     # CCup5foFAzXqbfSie
    "id.orig_h",               # 152.101.118.11
    "id.orig_p",               # 54932
    "id.resp_h",               # 172.31.69.6
    "id.resp_p",               # 3389
    "version",                 # TLSv10
    "cipher",                  # TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA
    "curve",                   # secp256r1
    "server_name",             # -
    "resumed",                 # F
    "last_alert",              # -.1
    "next_protocol",           # -.2
    "established",             # T
    "cert_chain_fuids",        # CsxknGIi
    "client_cert_chain_fuids", # hash
    "subject",                 # (empty)
    "issuer",                  # -.3
    "client_subject",          # self signed certificate
    "client_issuer",           # (may be similar)
    "validation_status",       # e.g. 'self signed certificate'
    "source_log"
]

print("\nssl_df cols BEFORE:", ssl_df.columns.tolist())
if len(ssl_df.columns) == len(ssl_cols):
    ssl_df.columns = ssl_cols
else:
    print("⚠️ ssl_df column count mismatch; got", len(ssl_df.columns), "expected", len(ssl_cols))

print("ssl_df cols AFTER:", ssl_df.columns.tolist())
display(ssl_df.head())


conn_df cols BEFORE: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 'source_log']
⚠️ conn_df column count mismatch; got 23 expected 22
conn_df cols AFTER: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 'source_log']


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,source_log
0,1.518697e+09,CnIDHq42unWALrSlp2,175.195.219.31,54576,172.31.69.30,3389,tcp,-,1.607211,746,...,T,0,DdaR,3,866,3,450,-,6,conn.log
1,1.518697e+09,CfDSEFzzaZlJPcG55,201.92.155.17,44655,172.31.68.21,81,tcp,-,-,-,...,T,0,S,1,44,0,0,-,6,conn.log
2,1.518697e+09,CmdO0d4pOoxS3JGAHh,175.195.219.31,55135,172.31.66.122,3389,tcp,-,2.328895,1085,...,T,0,DdaR,5,1285,5,1762,-,6,conn.log
3,1.518697e+09,C11P8c2rLsp4ZfuEd,172.31.66.122,49688,205.186.147.63,443,tcp,-,0.011620,0,...,F,0,Sr,1,52,1,40,-,6,conn.log
4,1.518697e+09,CA9dvAq5ATKNC5wNl,172.31.66.122,49688,205.186.147.63,443,tcp,-,0.011671,0,...,F,0,Sr,1,52,1,40,-,6,conn.log



ssl_df cols BEFORE: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 'source_log']
⚠️ ssl_df column count mismatch; got 20 expected 22
ssl_df cols AFTER: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 'source_log']


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,source_log
0,1.518697e+09,CCup5foFAzXqbfSie,152.101.118.11,54932,172.31.69.6,3389,TLSv10,TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA,secp256r1,-,F,-,-,T,CsxknGIi,57550dc9aa621ac2378ecc637a6a61d00924cfed56c866...,(empty),-,self signed certificate,ssl.log
1,1.518697e+09,CPJaA31ybvXxZQ53gh,172.31.65.118,49687,52.84.140.121,443,TLSv12,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,secp256r1,getpocket.cdn.mozilla.net,F,-,h2,T,CsxuknGIti,6442cb8d30d303bc67c685ba319e9497aa39aeffc3caca...,(empty),T,ok,ssl.log
2,1.518697e+09,CtqhVw1sdFNwzHePA,172.31.65.118,49686,52.84.140.121,443,TLSv12,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,secp256r1,getpocket.cdn.mozilla.net,F,-,h2,T,CsxuknGIti,6442cb8d30d303bc67c685ba319e9497aa39aeffc3caca...,(empty),T,ok,ssl.log
3,1.518697e+09,C2UFf21hzIrAB4q0W2,172.31.68.21,49691,54.192.193.78,443,TLSv12,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,secp256r1,getpocket.cdn.mozilla.net,F,-,h2,T,CsxuknGIti,6442cb8d30d303bc67c685ba319e9497aa39aeffc3caca...,(empty),T,ok,ssl.log
4,1.518697e+09,CiSc8vSu0grz0nSij,172.31.68.21,49692,54.192.193.78,443,TLSv12,TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,secp256r1,getpocket.cdn.mozilla.net,F,-,h2,T,CsxuknGIti,6442cb8d30d303bc67c685ba319e9497aa39aeffc3caca...,(empty),T,ok,ssl.log


In [5]:
# Make sure conn_df and ssl_df both have a 'uid' column

# For conn_df: take the 2nd column as uid if 'uid' doesn't exist
if "uid" not in conn_df.columns:
    # safety: ensure at least 2 columns exist
    if conn_df.shape[1] >= 2:
        conn_df["uid"] = conn_df.iloc[:, 1].astype(str)
    else:
        raise ValueError("conn_df does not have at least 2 columns to extract a uid from.")

# For ssl_df: same idea
if "uid" not in ssl_df.columns:
    if ssl_df.shape[1] >= 2:
        ssl_df["uid"] = ssl_df.iloc[:, 1].astype(str)
    else:
        raise ValueError("ssl_df does not have at least 2 columns to extract a uid from.")

print("conn_df columns:", conn_df.columns.tolist())
print("ssl_df columns:", ssl_df.columns.tolist())
print("Example conn_df row:\n", conn_df.head(1))
print("Example ssl_df row:\n", ssl_df.head(1))


conn_df columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 'source_log', 'uid']
ssl_df columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 'source_log', 'uid']
Example conn_df row:
               0                   1               2      3             4  \
0  1.518697e+09  CnIDHq42unWALrSlp2  175.195.219.31  54576  172.31.69.30   

      5    6  7         8    9  ... 14    15 16   17  18   19  20  21  \
0  3389  tcp  -  1.607211  746  ...  0  DdaR  3  866   3  450   -   6   

   source_log                 uid  
0    conn.log  CnIDHq42unWALrSlp2  

[1 rows x 24 columns]
Example ssl_df row:
               0                  1               2      3            4     5  \
0  1.518697e+09  CCup5foFAzXqbfSie  152.101.118.11  54932  172.31.69.6  3389   

        6                                   7          8  9  ... 11 12 13  \
0  TLSv10  TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA  secp256r1  -  ...  -  -  T   

         14              

In [6]:
# Merge conn and ssl logs on uid
if conn_df.empty or ssl_df.empty:
    raise ValueError("conn_df or ssl_df is empty. Check that both conn and ssl logs were loaded.")

sessions = conn_df.merge(
    ssl_df,
    on="uid",
    how="inner",
    suffixes=("_conn", "_ssl")
)

print("🔗 Merged sessions:", len(sessions))
sessions.head()


🔗 Merged sessions: 419903


,0_conn,1_conn,2_conn,3_conn,4_conn,5_conn,6_conn,7_conn,8_conn,9_conn,...,10_ssl,11_ssl,12_ssl,13_ssl,14_ssl,15_ssl,16_ssl,17_ssl,18_ssl,source_log_ssl
0,1.518697e+09,CPJaA31ybvXxZQ53gh,172.31.65.118,49687,52.84.140.121,443,tcp,ssl,0.235417,553,...,F,-,h2,T,CsxuknGIti,6442cb8d30d303bc67c685ba319e9497aa39aeffc3caca...,(empty),T,ok,ssl.log
1,1.518697e+09,CPgAK81tzKgHVlwSWe,172.31.68.21,49690,64.30.228.118,443,tcp,ssl,0.147160,194,...,F,unrecognized_name,-,F,Cl,-,-,-,-,ssl.log
2,1.518697e+09,C2UFf21hzIrAB4q0W2,172.31.68.21,49691,54.192.193.78,443,tcp,ssl,0.156903,553,...,F,-,h2,T,CsxuknGIti,6442cb8d30d303bc67c685ba319e9497aa39aeffc3caca...,(empty),T,ok,ssl.log
3,1.518697e+09,CJ6ZxI1RWTh1HZx5vi,172.31.69.30,49684,54.192.16.165,443,tcp,ssl,0.122010,553,...,F,-,h2,T,CsxuknGIti,6442cb8d30d303bc67c685ba319e9497aa39aeffc3caca...,(empty),T,ok,ssl.log
4,1.518697e+09,CwV1CU2c4B5oGaCTFb,172.31.68.21,49693,64.30.228.118,443,tcp,ssl,0.148246,194,...,F,unrecognized_name,-,F,Cl,-,-,-,-,ssl.log


In [7]:
# Fix conn-related columns after merge
conn_col_map = {
    0: "ts_conn",
    1: "uid",
    2: "id.orig_h",
    3: "id.orig_p",
    4: "id.resp_h",
    5: "id.resp_p",
    6: "proto",
    7: "service",
    8: "duration",
    9: "orig_bytes",
    10: "resp_bytes",
    11: "conn_state",
    12: "local_orig",
    13: "local_resp",
    14: "missed_bytes",
    15: "history",
    16: "orig_pkts",
    17: "orig_ip_bytes",
    18: "resp_pkts",
    19: "resp_ip_bytes",
    20: "tunnel_parents"
}
# Convert all column names to strings so we can safely use .endswith()
sessions.columns = sessions.columns.astype(str)

# Apply renaming only to conn columns in merged DataFrame
for idx, new_name in conn_col_map.items():
    if idx < len(sessions.columns):
        sessions.rename(columns={sessions.columns[idx]: new_name}, inplace=True)

# SSL columns (after merge appear later)
ssl_col_map = {
    "ts_ssl": "ts_ssl",
    "version": "version",
    "cipher": "cipher",
    "curve": "curve",
    "server_name": "server_name",
    "resumed": "resumed",
    "validation_status": "validation_status"
}

# Attempt to rename ssl based on known merge suffixes
for c in list(sessions.columns):
    if c.endswith("_ssl"):
        base = c.replace("_ssl", "")
        if base in ssl_col_map:
            sessions.rename(columns={c: ssl_col_map[base]}, inplace=True)


In [8]:
features = sessions.copy()

# Convert numeric columns
numeric_cols = []
for col in ["duration", "orig_bytes", "resp_bytes", "missed_bytes",
            "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]:
    if col in features.columns:
        numeric_cols.append(col)
        features[col] = pd.to_numeric(features[col], errors="coerce").fillna(0)

print("Numeric cols converted:", numeric_cols)
display(features[numeric_cols].describe())

# Feature engineering

# Total bytes
features["bytes_sum"] = features.get("orig_bytes", 0) + features.get("resp_bytes", 0)

# Ratio of bytes
features["bytes_ratio"] = features.get("orig_bytes", 0) / (features.get("resp_bytes", 0) + 1)

# Log duration
if "duration" in features.columns:
    features["duration_log"] = np.log1p(features["duration"])
else:
    features["duration_log"] = 0

# TLS version numeric
version_map = {
    "TLSv1": 1.0,
    "TLSv10": 1.0,   # some Zeek logs use TLSv10/TLSv11 strings
    "TLSv11": 1.1,
    "TLSv12": 1.2,
    "TLSv13": 1.3,
}
if "version" in features.columns:
    features["tls_version_num"] = features["version"].map(version_map).fillna(0)
else:
    features["tls_version_num"] = 0

# Resumption flag
if "resumed" in features.columns:
    features["resumed_flag"] = (features["resumed"] == "T").astype(int)  # or == 1 if numeric
else:
    features["resumed_flag"] = 0

features[[
    "uid",
    "duration", "duration_log",
    "orig_bytes", "resp_bytes", "bytes_sum", "bytes_ratio",
    "tls_version_num", "resumed_flag"
]].head()


Numeric cols converted: ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes']


,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes
count,419903.000000,4.199030e+05,4.199030e+05,4.199030e+05,419903.000000,419903.000000,419903.000000,4.199030e+05
mean,54.805519,2.921843e+03,7.222789e+03,9.603262e+02,13.710678,1840.360812,15.030743,7.006251e+03
std,145.550334,7.008976e+04,1.725076e+05,1.302509e+05,28.548012,3688.914821,61.654196,7.547406e+04
min,0.009018,8.800000e+01,0.000000e+00,0.000000e+00,3.000000,252.000000,1.000000,4.400000e+01
25%,2.425876,1.052000e+03,1.581000e+03,0.000000e+00,8.000000,1460.000000,7.000000,1.873000e+03
50%,8.759123,1.144000e+03,1.727000e+03,0.000000e+00,10.000000,1556.000000,10.000000,2.143000e+03
75%,90.123228,1.448000e+03,3.950000e+03,0.000000e+00,14.000000,1896.000000,13.000000,4.524000e+03
max,21062.889113,6.384896e+06,2.765908e+07,2.478241e+07,8499.000000,340651.000000,18074.000000,2.665487e+07


,uid,uid,duration,duration_log,orig_bytes,resp_bytes,bytes_sum,bytes_ratio,tls_version_num,resumed_flag
0,CPJaA31ybvXxZQ53gh,CPJaA31ybvXxZQ53gh,0.235417,0.211409,553,3811,4364,0.145068,0,0
1,CPgAK81tzKgHVlwSWe,CPgAK81tzKgHVlwSWe,0.147160,0.137289,194,7,201,24.250000,0,0
2,C2UFf21hzIrAB4q0W2,C2UFf21hzIrAB4q0W2,0.156903,0.145747,553,3811,4364,0.145068,0,0
3,CJ6ZxI1RWTh1HZx5vi,CJ6ZxI1RWTh1HZx5vi,0.122010,0.115122,553,3811,4364,0.145068,0,0
4,CwV1CU2c4B5oGaCTFb,CwV1CU2c4B5oGaCTFb,0.148246,0.138236,194,7,201,24.250000,0,0


In [9]:
# Basic export schema for Elastic / Kibana or ML

candidate_cols = [
    "uid",
    "ts_conn" if "ts_conn" in features.columns else "ts_conn",  # will fix next
    "ts_conn", "ts_ssl",
    "id.orig_h_conn", "id.orig_p_conn",
    "id.resp_h_conn", "id.resp_p_conn",
    "proto",
    "duration",
    "orig_bytes", "resp_bytes",
    "bytes_sum", "bytes_ratio",
    "version",
    "cipher",
    "server_name",
    "tls_version_num",
    "resumed_flag",
]

# Fix timestamp columns if they aren't split
if "ts_conn" not in features.columns and "ts_x" in features.columns:
    features.rename(columns={"ts_x": "ts_conn"}, inplace=True)
if "ts_ssl" not in features.columns and "ts_y" in features.columns:
    features.rename(columns={"ts_y": "ts_ssl"}, inplace=True)

# Build final export column list based on what actually exists
export_cols = [c for c in [
    "uid",
    "ts_conn" if "ts_conn" in features.columns else "ts",
    "id.orig_h_conn" if "id.orig_h_conn" in features.columns else "id.orig_h",
    "id.orig_p_conn" if "id.orig_p_conn" in features.columns else "id.orig_p",
    "id.resp_h_conn" if "id.resp_h_conn" in features.columns else "id.resp_h",
    "id.resp_p_conn" if "id.resp_p_conn" in features.columns else "id.resp_p",
    "proto",
    "duration",
    "orig_bytes", "resp_bytes",
    "bytes_sum", "bytes_ratio",
    "version",
    "cipher",
    "server_name",
    "tls_version_num",
    "resumed_flag",
] if c in features.columns]

print("Exporting columns:", export_cols)

export_df = features[export_cols].copy()
export_df.to_csv("sessions_for_elastic.csv", index=False)

print("✅ Exported sessions_for_elastic.csv with", len(export_df), "rows")
export_df.head()


Exporting columns: ['uid', 'ts_conn', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'duration', 'orig_bytes', 'resp_bytes', 'bytes_sum', 'bytes_ratio', 'tls_version_num', 'resumed_flag']
✅ Exported sessions_for_elastic.csv with 419903 rows


,uid,uid,ts_conn,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,orig_bytes,resp_bytes,bytes_sum,bytes_ratio,tls_version_num,resumed_flag
0,CPJaA31ybvXxZQ53gh,CPJaA31ybvXxZQ53gh,1.518697e+09,172.31.65.118,49687,52.84.140.121,443,tcp,0.235417,553,3811,4364,0.145068,0,0
1,CPgAK81tzKgHVlwSWe,CPgAK81tzKgHVlwSWe,1.518697e+09,172.31.68.21,49690,64.30.228.118,443,tcp,0.147160,194,7,201,24.250000,0,0
2,C2UFf21hzIrAB4q0W2,C2UFf21hzIrAB4q0W2,1.518697e+09,172.31.68.21,49691,54.192.193.78,443,tcp,0.156903,553,3811,4364,0.145068,0,0
3,CJ6ZxI1RWTh1HZx5vi,CJ6ZxI1RWTh1HZx5vi,1.518697e+09,172.31.69.30,49684,54.192.16.165,443,tcp,0.122010,553,3811,4364,0.145068,0,0
4,CwV1CU2c4B5oGaCTFb,CwV1CU2c4B5oGaCTFb,1.518697e+09,172.31.68.21,49693,64.30.228.118,443,tcp,0.148246,194,7,201,24.250000,0,0


In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv("sessions_for_elastic.csv")
print("Loaded:", df.shape)
df.head()


Loaded: (419903, 15)


,uid,uid.1,ts_conn,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,orig_bytes,resp_bytes,bytes_sum,bytes_ratio,tls_version_num,resumed_flag
0,CPJaA31ybvXxZQ53gh,CPJaA31ybvXxZQ53gh,1.518697e+09,172.31.65.118,49687,52.84.140.121,443,tcp,0.235417,553,3811,4364,0.145068,0,0
1,CPgAK81tzKgHVlwSWe,CPgAK81tzKgHVlwSWe,1.518697e+09,172.31.68.21,49690,64.30.228.118,443,tcp,0.147160,194,7,201,24.250000,0,0
2,C2UFf21hzIrAB4q0W2,C2UFf21hzIrAB4q0W2,1.518697e+09,172.31.68.21,49691,54.192.193.78,443,tcp,0.156903,553,3811,4364,0.145068,0,0
3,CJ6ZxI1RWTh1HZx5vi,CJ6ZxI1RWTh1HZx5vi,1.518697e+09,172.31.69.30,49684,54.192.16.165,443,tcp,0.122010,553,3811,4364,0.145068,0,0
4,CwV1CU2c4B5oGaCTFb,CwV1CU2c4B5oGaCTFb,1.518697e+09,172.31.68.21,49693,64.30.228.118,443,tcp,0.148246,194,7,201,24.250000,0,0


In [11]:
# --- Add engineered features ---

df["duration_log"] = np.log1p(df["duration"])

df["byte_diff"] = df["orig_bytes"] - df["resp_bytes"]

df["byte_ratio2"] = df["resp_bytes"] / (df["orig_bytes"] + 1)

df["flow_entropy"] = (
    (df["orig_bytes"] / (df["bytes_sum"] + 1)) * np.log1p(df["orig_bytes"]) +
    (df["resp_bytes"] / (df["bytes_sum"] + 1)) * np.log1p(df["resp_bytes"])
)

df["is_tls"] = (df["tls_version_num"] > 0).astype(int)

df["burstiness"] = df["bytes_sum"] / (df["duration"] + 0.001)

df.head()


,uid,uid.1,ts_conn,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,orig_bytes,...,bytes_sum,bytes_ratio,tls_version_num,resumed_flag,duration_log,byte_diff,byte_ratio2,flow_entropy,is_tls,burstiness
0,CPJaA31ybvXxZQ53gh,CPJaA31ybvXxZQ53gh,1.518697e+09,172.31.65.118,49687,52.84.140.121,443,tcp,0.235417,553,...,4364,0.145068,0,0,0.211409,-3258,6.879061,7.999668,0,18458.909469
1,CPgAK81tzKgHVlwSWe,CPgAK81tzKgHVlwSWe,1.518697e+09,172.31.68.21,49690,64.30.228.118,443,tcp,0.147160,194,...,201,24.250000,0,0,0.137289,187,0.035897,5.136228,0,1356.641469
2,C2UFf21hzIrAB4q0W2,C2UFf21hzIrAB4q0W2,1.518697e+09,172.31.68.21,49691,54.192.193.78,443,tcp,0.156903,553,...,4364,0.145068,0,0,0.145747,-3258,6.879061,7.999668,0,27637.220319
3,CJ6ZxI1RWTh1HZx5vi,CJ6ZxI1RWTh1HZx5vi,1.518697e+09,172.31.69.30,49684,54.192.16.165,443,tcp,0.122010,553,...,4364,0.145068,0,0,0.115122,-3258,6.879061,7.999668,0,35476.790505
4,CwV1CU2c4B5oGaCTFb,CwV1CU2c4B5oGaCTFb,1.518697e+09,172.31.68.21,49693,64.30.228.118,443,tcp,0.148246,194,...,201,24.250000,0,0,0.138236,187,0.035897,5.136228,0,1346.769763


In [12]:
import numpy as np

# --- Create a copy so we don't modify the original df accidentally ---
df_labeled = df.copy()

# --- Initialize all flows as benign ---
df_labeled["label"] = "Benign"


In [13]:
df_labeled.loc[df_labeled["id.resp_p"] == 3389, "label"] = "RDP-Bruteforce"


In [14]:
df_labeled.loc[df_labeled["tls_version_num"] < 1.2, "label"] = "Suspicious-TLS"


In [15]:
df_labeled.loc[df_labeled["burstiness"] > df_labeled["burstiness"].quantile(0.99),
               "label"] = "High-Burstiness"


In [16]:
df_labeled.loc[df_labeled["bytes_ratio"] > 50, "label"] = "Data-Exfiltration"


In [17]:
mask = (df_labeled["duration"] < 0.5) & (df_labeled["bytes_sum"] > 20000)
df_labeled.loc[mask, "label"] = "Exploit-Delivery"


In [18]:
df_labeled.loc[df_labeled["flow_entropy"] > df_labeled["flow_entropy"].quantile(0.98),
               "label"] = "High-Entropy"


In [19]:
suspicious_prefixes = [
    "45.", "46.", "77.", "91.", "109.", "136.", "176.", "185.", "188.", "193."
]

df_labeled.loc[
    df_labeled["id.resp_h"].astype(str).str.startswith(tuple(suspicious_prefixes)),
    "label"
] = "Suspicious-IP"


In [20]:
df_labeled["label"].value_counts()


,count
label,
Suspicious-TLS,406292
High-Entropy,8365
High-Burstiness,3583
Suspicious-IP,1210
Exploit-Delivery,311
Data-Exfiltration,142


In [21]:
df_labeled.to_csv("heuristic_labeled_sessions.csv", index=False)
df_labeled.to_json("elastic_heuristic.json", orient="records", lines=True)

print("Exported heuristic-labeled datasets!")


Exported heuristic-labeled datasets!


In [22]:
import numpy as np
import pandas as pd
import os

# 1. Quick sanity check
print("sessions shape:", sessions.shape)
print("Current columns:\n", sessions.columns.tolist())

# 2. Make sure the core Zeek fields we expect are there
base_required = [
    "uid",
    "ts_conn",
    "id.orig_h",
    "id.orig_p",
    "id.resp_h",
    "id.resp_p",
    "proto",
    "duration",
    "orig_bytes",
    "resp_bytes",
]

missing_base = [c for c in base_required if c not in sessions.columns]
if missing_base:
    raise ValueError(f"These *base* columns are missing from sessions: {missing_base}")

# 3. Create engineered columns if they’re missing

# bytes_sum = orig_bytes + resp_bytes
if "bytes_sum" not in sessions.columns:
    ob = pd.to_numeric(sessions["orig_bytes"], errors="coerce").fillna(0)
    rb = pd.to_numeric(sessions["resp_bytes"], errors="coerce").fillna(0)
    sessions["bytes_sum"] = ob + rb
    print("Added bytes_sum")

# bytes_ratio = orig_bytes / resp_bytes (safe division)
if "bytes_ratio" not in sessions.columns:
    ob = pd.to_numeric(sessions["orig_bytes"], errors="coerce").fillna(0)
    rb = pd.to_numeric(sessions["resp_bytes"], errors="coerce").fillna(0)

    # avoid division by zero
    rb_safe = rb.replace(0, np.nan)
    ratio = ob / rb_safe
    sessions["bytes_ratio"] = ratio.fillna(0.0)
    print("Added bytes_ratio")

# tls_version_num – try to derive from any TLS/SSL version column, else 0
if "tls_version_num" not in sessions.columns:
    made_tls = False
    for cand in ["tls.version", "ssl.version", "version"]:
        if cand in sessions.columns:
            v = sessions[cand].astype(str).str.extract(r"(\d+)").iloc[:, 0]
            sessions["tls_version_num"] = pd.to_numeric(v, errors="coerce").fillna(0).astype(int)
            print(f"Derived tls_version_num from {cand}")
            made_tls = True
            break
    if not made_tls:
        sessions["tls_version_num"] = 0
        print("No TLS version column found → tls_version_num set to 0")

# resumed_flag – from ssl.resumed/resumed if present, else 0
if "resumed_flag" not in sessions.columns:
    made_resumed = False
    for cand in ["ssl.resumed", "resumed"]:
        if cand in sessions.columns:
            sessions["resumed_flag"] = (sessions[cand].astype(str) == "T").astype(int)
            print(f"Derived resumed_flag from {cand}")
            made_resumed = True
            break
    if not made_resumed:
        sessions["resumed_flag"] = 0
        print("No resumed column found → resumed_flag set to 0")

# 4. Now export with the full list
export_cols = [
    "uid",
    "ts_conn",
    "id.orig_h",
    "id.orig_p",
    "id.resp_h",
    "id.resp_p",
    "proto",
    "duration",
    "orig_bytes",
    "resp_bytes",
    "bytes_sum",
    "bytes_ratio",
    "tls_version_num",
    "resumed_flag",
]

df_export = sessions[export_cols].copy()
df_export.to_csv("sessions_for_elastic.csv", index=False)

print("\n✅ Exported sessions_for_elastic.csv")
print("Path:", os.path.abspath("sessions_for_elastic.csv"))


sessions shape: (419903, 44)
Current columns:
 ['ts_conn', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents', '21', 'source_log_conn', 'uid', '0_ssl', '1_ssl', '2_ssl', '3_ssl', '4_ssl', '5_ssl', '6_ssl', '7_ssl', '8_ssl', '9_ssl', '10_ssl', '11_ssl', '12_ssl', '13_ssl', '14_ssl', '15_ssl', '16_ssl', '17_ssl', '18_ssl', 'source_log_ssl']
Added bytes_sum
Added bytes_ratio
No TLS version column found → tls_version_num set to 0
No resumed column found → resumed_flag set to 0

✅ Exported sessions_for_elastic.csv
Path: /content/sessions_for_elastic.csv


In [23]:
from google.colab import files
files.download("sessions_for_elastic.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
import os

print("Current working directory:", os.getcwd())
print("Files here:")
print(os.listdir("."))



Current working directory: /content
Files here:
['.config', 'heuristic_labeled_sessions.csv', 'zeek_logs.zip', 'elastic_heuristic.json', 'zeek_logs', 'sessions_for_elastic.csv', 'sample_data']


In [25]:
import os

def find_file(name, path="."):
    hits = []
    for root, dirs, files in os.walk(path):
        if name in files:
            hits.append(os.path.join(root, name))
    return hits

print("Searching for sessions_for_elastic.csv...")
matches = find_file("sessions_for_elastic.csv", "/")
matches[:10]  # show first 10 matches


Searching for sessions_for_elastic.csv...


['/content/sessions_for_elastic.csv']

In [26]:
from google.colab import files
uploaded = files.upload()

import pandas as pd

filename = list(uploaded.keys())[0]
print("Loaded file:", filename)

df = pd.read_csv(filename)
df.head()


Saving sessions_for_elastic.csv to sessions_for_elastic (1).csv
Loaded file: sessions_for_elastic (1).csv


,uid,uid.1,ts_conn,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,orig_bytes,resp_bytes,bytes_sum,bytes_ratio,tls_version_num,resumed_flag
0,CPJaA31ybvXxZQ53gh,CPJaA31ybvXxZQ53gh,1.518697e+09,172.31.65.118,49687,52.84.140.121,443,tcp,0.235417,553,3811,4364,0.145106,0,0
1,CPgAK81tzKgHVlwSWe,CPgAK81tzKgHVlwSWe,1.518697e+09,172.31.68.21,49690,64.30.228.118,443,tcp,0.147160,194,7,201,27.714286,0,0
2,C2UFf21hzIrAB4q0W2,C2UFf21hzIrAB4q0W2,1.518697e+09,172.31.68.21,49691,54.192.193.78,443,tcp,0.156903,553,3811,4364,0.145106,0,0
3,CJ6ZxI1RWTh1HZx5vi,CJ6ZxI1RWTh1HZx5vi,1.518697e+09,172.31.69.30,49684,54.192.16.165,443,tcp,0.122010,553,3811,4364,0.145106,0,0
4,CwV1CU2c4B5oGaCTFb,CwV1CU2c4B5oGaCTFb,1.518697e+09,172.31.68.21,49693,64.30.228.118,443,tcp,0.148246,194,7,201,27.714286,0,0


In [27]:
import os

for root, dirs, files in os.walk("/", topdown=True):
    for name in files:
        if "elastic" in name or name.endswith(".csv"):
            print(os.path.join(root, name))


/proc/sys/net/ipv6/route/gc_elasticity
/root/.julia/packages/CSV/XLcqT/test/out.test.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/test_comments_multichar.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/int64_overflow.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/dash_as_null.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/types_override.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/test_single_column.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/test_2_footer_rows.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/test_not_enough_columns2.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/test_truestrings.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/transposed.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/test_comments1.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/test_correct_trailing_missings.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/FL_insurance_sample.csv
/root/.julia/packages/CSV/XLcqT/test/testfiles/test_types.csv
/root/.julia/

In [28]:
# Make df the main feature table we’re working with
df = features.copy()  # or df = sessions.copy() if you built features directly there


In [29]:
import pandas as pd
import numpy as np

# If sessions exists in memory use it, otherwise load merged CSV
try:
    df_base = sessions.copy()
except:
    df_base = pd.read_csv("sessions_for_elastic.csv")

df_labeled = df_base.copy()

# Ensure numeric
for col in ["duration", "orig_bytes", "resp_bytes"]:
    if col in df_labeled.columns:
        df_labeled[col] = pd.to_numeric(df_labeled[col], errors="coerce")

# Feature engineering
df_labeled["bytes_sum"] = (
    df_labeled["orig_bytes"].fillna(0) + df_labeled["resp_bytes"].fillna(0)
)

df_labeled["bytes_ratio"] = (
    df_labeled["orig_bytes"] /
    df_labeled["resp_bytes"].replace({0: np.nan})
)
df_labeled["bytes_ratio"] = df_labeled["bytes_ratio"].fillna(0)

# Create label column
df_labeled["label"] = "Benign"

q_dur = df_labeled["duration"].quantile(0.95)
q_bytes = df_labeled["bytes_sum"].quantile(0.95)

malicious_mask = (
    (df_labeled["duration"] > q_dur) |
    (df_labeled["bytes_sum"] > q_bytes)
)

df_labeled.loc[malicious_mask, "label"] = "Malicious"
df_labeled["is_malicious"] = (df_labeled["label"] == "Malicious").astype(int)

print("Label counts:", df_labeled["label"].value_counts())
print("Binary counts:", df_labeled["is_malicious"].value_counts())


Label counts: label
Benign       385927
Malicious     33976
Name: count, dtype: int64
Binary counts: is_malicious
0    385927
1     33976
Name: count, dtype: int64


In [30]:
# Feature columns used for ML training
feature_cols = [
    "duration",
    "orig_bytes",
    "resp_bytes",
    "bytes_sum",
    "bytes_ratio",
    "tls_version_num",
    "resumed_flag",
]

# Only include columns that actually exist in df
feature_cols = [c for c in feature_cols if c in df.columns]

print("Using feature columns:", feature_cols)


Using feature columns: ['duration', 'orig_bytes', 'resp_bytes', 'bytes_sum', 'bytes_ratio', 'tls_version_num', 'resumed_flag']


In [31]:
import numpy as np

# 1) Start from your merged sessions DataFrame
df_labeled = sessions.copy()

# 2) Make sure key numeric columns are actually numeric
numeric_base_cols = ["duration", "orig_bytes", "resp_bytes"]

for col in numeric_base_cols:
    if col in df_labeled.columns:
        df_labeled[col] = pd.to_numeric(df_labeled[col], errors="coerce")
    else:
        print(f"WARNING: column {col} not found in df_labeled.columns")

# 3) Create bytes_sum and bytes_ratio safely
if "orig_bytes" in df_labeled.columns and "resp_bytes" in df_labeled.columns:
    df_labeled["bytes_sum"] = (
        df_labeled["orig_bytes"].fillna(0) + df_labeled["resp_bytes"].fillna(0)
    )

    # Avoid divide-by-zero
    df_labeled["bytes_ratio"] = df_labeled["orig_bytes"] / (
        df_labeled["resp_bytes"].replace({0: np.nan})
    )
    df_labeled["bytes_ratio"] = df_labeled["bytes_ratio"].fillna(0)
else:
    # fallback if something is weird
    df_labeled["bytes_sum"] = 0.0
    df_labeled["bytes_ratio"] = 0.0

# 4) Initialize everything as BENIGN
df_labeled["label"] = "Benign"

# 5) Compute quantiles (now that they're numeric)
q_dur = df_labeled["duration"].quantile(0.95)
q_bytes = df_labeled["bytes_sum"].quantile(0.95)

# 6) Mark "malicious" using a simple heuristic
malicious_mask = (
    (df_labeled["duration"] > q_dur) |
    (df_labeled["bytes_sum"] > q_bytes)
)

df_labeled.loc[malicious_mask, "label"] = "Malicious"

# 7) Binary label
df_labeled["is_malicious"] = (df_labeled["label"] == "Malicious").astype(int)

print("Label counts:")
print(df_labeled["label"].value_counts(dropna=False))

print("\nBinary label counts (is_malicious):")
print(df_labeled["is_malicious"].value_counts(dropna=False))


Label counts:
label
Benign       385927
Malicious     33976
Name: count, dtype: int64

Binary label counts (is_malicious):
is_malicious
0    385927
1     33976
Name: count, dtype: int64


In [32]:
import pandas as pd
import numpy as np

# -----------------------------------------------------
# Load source data from whichever variable exists
# -----------------------------------------------------
if "sessions" in globals():
    df_labeled = sessions.copy()
    print("Loaded from sessions variable:", df_labeled.shape)
else:
    df_labeled = pd.read_csv("sessions_for_elastic.csv")
    print("Loaded from CSV:", df_labeled.shape)

# -----------------------------------------------------
# Ensure required numeric columns exist
# -----------------------------------------------------
for col in ["duration", "orig_bytes", "resp_bytes"]:
    if col not in df_labeled.columns:
        df_labeled[col] = 0
    df_labeled[col] = pd.to_numeric(df_labeled[col], errors="coerce").fillna(0)

# -----------------------------------------------------
# Create derived numeric fields
# -----------------------------------------------------
df_labeled["bytes_sum"] = (
    df_labeled["orig_bytes"].fillna(0) + df_labeled["resp_bytes"].fillna(0)
)

df_labeled["bytes_ratio"] = df_labeled["orig_bytes"] / \
    df_labeled["resp_bytes"].replace({0: np.nan})

df_labeled["bytes_ratio"] = df_labeled["bytes_ratio"].fillna(0)

# -----------------------------------------------------
# Create labels (Benign by default)
# -----------------------------------------------------
df_labeled["label"] = "Benign"

q_dur = df_labeled["duration"].quantile(0.95)
q_bytes = df_labeled["bytes_sum"].quantile(0.95)

malicious_mask = (
    (df_labeled["duration"] > q_dur) |
    (df_labeled["bytes_sum"] > q_bytes)
)

df_labeled.loc[malicious_mask, "label"] = "Malicious"

# -----------------------------------------------------
# Create binary label is_malicious
# -----------------------------------------------------
df_labeled["is_malicious"] = (df_labeled["label"] == "Malicious").astype(int)

print("\n===== LABEL SUMMARY =====")
print(df_labeled["label"].value_counts())
print(df_labeled["is_malicious"].value_counts())
print("==========================")


Loaded from sessions variable: (419903, 48)

===== LABEL SUMMARY =====
label
Benign       385927
Malicious     33976
Name: count, dtype: int64
is_malicious
0    385927
1     33976
Name: count, dtype: int64


In [33]:
# ==========================================================
# FULL CLEAN TRAINING + PREDICTION + EXPORT PIPELINE
# ==========================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

print("Starting ML pipeline...")

# ----------------------------------------------------------
# 1. DEFINE FEATURES (use ONLY columns that actually exist)
# ----------------------------------------------------------
feature_cols = [
    "duration",
    "orig_bytes",
    "resp_bytes",
    "bytes_sum",
    "bytes_ratio",
    "pkts_sum",
    "bytes_per_sec",
    "pkts_per_sec",
    "tls_version_num",
    "resumed_flag"
]

# Check which exist
existing = [c for c in feature_cols if c in df_labeled.columns]
missing = [c for c in feature_cols if c not in df_labeled.columns]

print("Using features:", existing)
print("Missing features removed:", missing)

# Final feature list
feature_cols = existing

# ----------------------------------------------------------
# 2. BUILD X, y
# ----------------------------------------------------------
X = df_labeled[feature_cols]
y = df_labeled["is_malicious"].astype(int)

# ----------------------------------------------------------
# 3. SPLIT TRAIN/TEST
# ----------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# ----------------------------------------------------------
# 4. TRAIN RANDOM FOREST
# ----------------------------------------------------------
rf = RandomForestClassifier(
    n_estimators=150,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
print("Model trained!")

# ----------------------------------------------------------
# 5. EVALUATE
# ----------------------------------------------------------
y_pred = rf.predict(X_test)

print("\n===== CLASSIFICATION REPORT =====")
print(classification_report(y_test, y_pred, target_names=["Benign", "Malicious"]))

print("\n===== CONFUSION MATRIX =====")
print(confusion_matrix(y_test, y_pred))

# ----------------------------------------------------------
# 6. PREDICT ON **ALL** FLOWS
# ----------------------------------------------------------
X_all = df_labeled[feature_cols]

df_labeled["rf_score"] = rf.predict_proba(X_all)[:, 1]
df_labeled["rf_pred"] = rf.predict(X_all).astype(int)

print("\nSample predictions:")
print(df_labeled[["label", "is_malicious", "rf_score", "rf_pred"]].head())

# ----------------------------------------------------------
# 7. EXPORT FOR ELASTICSEARCH
# ----------------------------------------------------------
export_cols = [
    "uid",
    "ts_conn",
    "id.orig_h",
    "id.orig_p",
    "id.resp_h",
    "id.resp_p",
    "proto",
    "duration",
    "orig_bytes",
    "resp_bytes",
    "bytes_sum",
    "bytes_ratio",
    "label",
    "is_malicious",
    "rf_score",
    "rf_pred"
]

# Keep only existing columns
export_cols = [c for c in export_cols if c in df_labeled.columns]

df_export = df_labeled[export_cols].copy()
df_export.to_csv("sessions_for_elastic.csv", index=False)

print("\n=====================================")
print("✅ Exported sessions_for_elastic.csv")
print("Rows:", len(df_export))
print("Columns:", export_cols)
print("=====================================")


Starting ML pipeline...
Using features: ['duration', 'orig_bytes', 'resp_bytes', 'bytes_sum', 'bytes_ratio', 'tls_version_num', 'resumed_flag']
Missing features removed: ['pkts_sum', 'bytes_per_sec', 'pkts_per_sec']
Model trained!

===== CLASSIFICATION REPORT =====
              precision    recall  f1-score   support

      Benign       1.00      1.00      1.00    115778
   Malicious       1.00      1.00      1.00     10193

    accuracy                           1.00    125971
   macro avg       1.00      1.00      1.00    125971
weighted avg       1.00      1.00      1.00    125971


===== CONFUSION MATRIX =====
[[115778      0]
 [     0  10193]]

Sample predictions:
    label  is_malicious  rf_score  rf_pred
0  Benign             0       0.0        0
1  Benign             0       0.0        0
2  Benign             0       0.0        0
3  Benign             0       0.0        0
4  Benign             0       0.0        0

✅ Exported sessions_for_elastic.csv
Rows: 419903
Columns: ['ui

In [39]:
import pandas as pd

# ------------------------------------------------
# 1) Get the main dataframe
#    Prefer df_labeled in memory; otherwise load the FINAL ML CSV
# ------------------------------------------------
if "df_labeled" in globals():
    df = df_labeled.copy()
    print("Using df_labeled from memory:", df.shape)
else:
    df = pd.read_csv("sessions_for_elastic_final_ml.csv")
    print("Loaded from CSV:", df.shape)

print("Current columns:", df.columns.tolist())

# ------------------------------------------------
# 2) Make sure we HAVE an is_malicious column
#    If it's missing, build it from label or rf_pred
# ------------------------------------------------
if "is_malicious" not in df.columns:
    if "label" in df.columns:
        # Assume label is 'Malicious' vs 'Benign'
        df["is_malicious"] = (df["label"] == "Malicious").astype(int)
        print("Created is_malicious from label")
    elif "rf_pred" in df.columns:
        # Binary prediction from the Random Forest
        df["is_malicious"] = df["rf_pred"].astype(int)
        print("Created is_malicious from rf_pred")
    else:
        raise ValueError(
            "No 'is_malicious', 'label', or 'rf_pred' column found – "
            "run the labeling + ML cells first."
        )

# ------------------------------------------------
# 3) Add threat_type based on simple rules
# ------------------------------------------------
df["threat_type"] = "Benign"
mask_mal = df["is_malicious"] == 1
df.loc[mask_mal, "threat_type"] = "Malicious-Other"

# Suspicious TLS: old TLS version
if "tls_version_num" in df.columns:
    df.loc[mask_mal & (df["tls_version_num"] < 1.2), "threat_type"] = "Suspicious-TLS"

# Data Exfiltration: huge outgoing ratio
if "bytes_ratio" in df.columns:
    df.loc[mask_mal & (df["bytes_ratio"] > 50), "threat_type"] = "Data-Exfiltration"

# Exploit Delivery: short duration and large bytes_sum
if all(col in df.columns for col in ["duration", "bytes_sum"]):
    df.loc[
        mask_mal &
        (df["duration"] < 0.5) &
        (df["bytes_sum"] > 20000),
        "threat_type"
    ] = "Exploit-Delivery"

# RDP Brute Force: traffic to port 3389
if "id.resp_p" in df.columns:
    df.loc[mask_mal & (df["id.resp_p"] == 3389), "threat_type"] = "RDP-Bruteforce"

print("\nThreat Type Breakdown:")
print(df["threat_type"].value_counts())

# ------------------------------------------------
# 4) Save updated CSV
# ------------------------------------------------
df.to_csv("sessions_for_elastic_final_ml_with_threat_type.csv", index=False)
print("\nSaved sessions_for_elastic_final_ml_with_threat_type.csv")


Using df_labeled from memory: (419903, 52)
Current columns: ['ts_conn', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents', '21', 'source_log_conn', 'uid', '0_ssl', '1_ssl', '2_ssl', '3_ssl', '4_ssl', '5_ssl', '6_ssl', '7_ssl', '8_ssl', '9_ssl', '10_ssl', '11_ssl', '12_ssl', '13_ssl', '14_ssl', '15_ssl', '16_ssl', '17_ssl', '18_ssl', 'source_log_ssl', 'bytes_sum', 'bytes_ratio', 'tls_version_num', 'resumed_flag', 'label', 'is_malicious', 'rf_score', 'rf_pred']

Threat Type Breakdown:
threat_type
Benign               385927
Suspicious-TLS        33360
Exploit-Delivery        441
Data-Exfiltration       173
RDP-Bruteforce            2
Name: count, dtype: int64

Saved sessions_for_elastic_final_ml_with_threat_type.csv


In [40]:
# Use the SAME feature_cols and trained rf as above

X_all = df_labeled[feature_cols]   # same columns as training

# Get model probability that a flow is malicious (class 1)
rf_probs = rf.predict_proba(X_all)[:, 1]

# Hard prediction from model (0 = benign, 1 = malicious)
rf_preds = rf.predict(X_all)

df_labeled["rf_score"] = rf_probs            # probability of malicious
df_labeled["rf_pred"]  = rf_preds.astype(int)

print(df_labeled[["label", "is_malicious", "rf_score", "rf_pred"]].head())


    label  is_malicious  rf_score  rf_pred
0  Benign             0       0.0        0
1  Benign             0       0.0        0
2  Benign             0       0.0        0
3  Benign             0       0.0        0
4  Benign             0       0.0        0


In [41]:
export_cols = [
    "uid",
    "ts_conn",
    "id.orig_h",
    "id.orig_p",
    "id.resp_h",
    "id.resp_p",
    "proto",
    "duration",
    "orig_bytes",
    "resp_bytes",
    "bytes_sum",
    "bytes_ratio",
    "label",         # heuristic label
    "is_malicious",  # heuristic 0/1
    "rf_score",      # model probability
    "rf_pred",       # model prediction
]

missing = [c for c in export_cols if c not in df_labeled.columns]
print("Missing columns:", missing)

df_export = df_labeled[[c for c in export_cols if c in df_labeled.columns]].copy()
df_export.to_csv("sessions_for_elastic.csv", index=False)

print("✅ Exported sessions_for_elastic.csv with shape:", df_export.shape)


Missing columns: []
✅ Exported sessions_for_elastic.csv with shape: (419903, 17)


In [43]:
import pandas as pd
import os

# Load your big CSV
df = pd.read_csv("sessions_for_elastic_final_ml_with_threat_type.csv")
print("Original shape:", df.shape)

# Keep only the columns that matter for analysis / dashboards
cols_keep = [
    "ts_conn",
    "uid",
    "id.orig_h", "id.orig_p",
    "id.resp_h", "id.resp_p",
    "proto",
    "service",
    "duration",
    "orig_bytes", "resp_bytes",
    "bytes_sum", "bytes_ratio",
    "tls_version_num", "resumed_flag",
    "label", "is_malicious",
    "rf_score", "rf_pred",
    "threat_type",
]

# Filter to only existing columns (just in case)
cols_keep = [c for c in cols_keep if c in df.columns]

df_small = df[cols_keep].copy()
print("Reduced shape:", df_small.shape)

# Save the smaller CSV
out_name = "sessions_for_elastic_small.csv"
df_small.to_csv(out_name, index=False)

size_mb = os.path.getsize(out_name) / (1024 * 1024)
print(f"{out_name} size: {size_mb:.2f} MB")


Original shape: (419903, 53)
Reduced shape: (419903, 20)
sessions_for_elastic_small.csv size: 60.94 MB


In [44]:
import pandas as pd
from google.colab import files

# ----------------------------------------------------------
# 1) PRE-ML EXPORT (features only)
# ----------------------------------------------------------

pre_cols = [
    "uid", "ts_conn",
    "id.orig_h", "id.orig_p",
    "id.resp_h", "id.resp_p",
    "proto",
    "duration",
    "orig_bytes", "resp_bytes",
    "bytes_sum", "bytes_ratio",
    "tls_version_num", "resumed_flag",
]

# Base dataframe (df_labeled if exists, else sessions)
if "df_labeled" in globals():
    base_df = df_labeled
elif "sessions" in globals():
    base_df = sessions
else:
    raise RuntimeError("Error: no dataframe found. Run merging + ML first.")

pre_existing = [c for c in pre_cols if c in base_df.columns]
pre_df = base_df[pre_existing].copy()

pre_filename = "sessions_for_elastic_pre.csv"
pre_df.to_csv(pre_filename, index=False)

print("✅ Created PRE-ML CSV:", pre_filename)
files.download(pre_filename)  # ⬅ automatic download


# ----------------------------------------------------------
# 2) FINAL ML EXPORT (features + labels + predictions)
# ----------------------------------------------------------

if "df_labeled" in globals():

    ml_cols = pre_cols + ["label", "is_malicious", "rf_score", "rf_pred"]
    ml_existing = [c for c in ml_cols if c in df_labeled.columns]

    ml_df = df_labeled[ml_existing].copy()

    ml_filename = "sessions_for_elastic_final_ml.csv"
    ml_df.to_csv(ml_filename, index=False)

    print("✅ Created FINAL ML CSV:", ml_filename)
    files.download(ml_filename)  # ⬅ automatic download

else:
    print("\n⚠️ Could not create ML CSV because df_labeled was not found.")


✅ Created PRE-ML CSV: sessions_for_elastic_pre.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Created FINAL ML CSV: sessions_for_elastic_final_ml.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>